# 進階的 SQL 五十道練習

> 資料定義語言

[數聚點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
%LOAD mysql user=root password=hahowsql

## （複習）SQL 的分類

- 在初階課程「SQL 的五十道練習」中我們專注在資料查詢語言的部分；這堂進階課程會完整帶學員們認識其他的 SQL 語言。
- 實務上，資料查詢語言是被分類在資料操作語言之下的一個分支。
    - **資料定義語言（Data Definition Language, DDL）**
    - 資料操作語言（Data Manipulation Language, DML）
        - 資料查詢語言（Data Query Language, DQL）
    - 資料控制語言（Data Control Language, DCL）
    - 交易控制語言（Transaction Control Language, TCL）

## 資料定義語言（Data Definition Language, DDL）主要的保留字

- `CREATE`
- `TRUNCATE`
- `DROP`
- `ALTER`

這意味著我們可以在關聯式資料庫管理系統中創造、刪除與更新物件。

## 有哪些物件可以被創造、刪除與更新

- 資料庫（Database）。
- 資料表（Table）。
- 檢視表（View）。
- 角色（Role）與使用者（User）。

## 資料庫（Database）

## 以 `CREATE DATABASE` 創造資料庫

- 執行之後可以在左側資料庫按右鍵下拉式選單點選 Refresh 即可檢視資料庫是否被創造。
- 或者使用 `SHOW DATABASES;` 指令檢視。

```sql
CREATE DATABASE database_name;
```

In [2]:
CREATE DATABASE cloned_imdb;

## 關聯式資料庫管理系統不允許相同命名的資料庫

如果我們再執行一次前述的 `CREATE DATABASE database_name;` 指令會得到錯誤。

## 以 `DROP DATABASE` 刪除資料庫

- 執行之後可以在左側資料庫按右鍵下拉式選單點選 Refresh 即可檢視資料庫是否被刪除。
- 或者使用 `SHOW DATABASES;` 指令檢視。

```sql
DROP DATABASE db_name;
```

In [3]:
DROP DATABASE cloned_imdb;

## 資料庫不存在時不允許刪除

如果我們再執行一次前述的 `DROP DATABASE database_name;` 指令會得到錯誤。

## 加入 `IF EXISTS`

在加入 `IF EXISTS` 的情況下，可以避免得到錯誤。

```sql
DROP DATABASE IF EXISTS database_name;
```

In [4]:
DROP DATABASE IF EXISTS cloned_imdb;

## 連線工具的使用者介面也可以輕鬆創造、刪除資料庫

- 左側資料庫按右鍵下拉式選單點選 Create New Database
- 選擇 Charset 與 Collation
    - Charset: utf8mb4
    - Collation: utf8mb4_unicode_ci
- 左側資料庫按右鍵下拉式選單點選 Delete

## 什麼是 Charset 與 Collation

- MySQL utf8 並不是一般常用的 utf8 編碼，而是 utf8mb3(most bytes 3)
- MySQL utf8mb4(most bytes 4)對應到一般常用的 utf8 編碼，可以顯示冷僻字或者表情符號（Emoji）
- Collation 影響文字資料的排序以及搜尋：utf8mb4_unicode_ci 準確但比較慢 utf8mb4_general_ci 速度快但相對不準確。

## 在 `CREATE DATABASE` 加入 Character Set 與 Collation

```sql
CREATE DATABASE database_name CHARACTER SET charset_name COLLATE collate_name;
```

In [5]:
CREATE DATABASE cloned_imdb CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

## 資料表（Table）

## 以 `CREATE TABLE` 創造資料表

- 執行之後可以在左側資料庫按右鍵下拉式選單點選 Refresh 即可檢視資料表是否被創造。
- 或者使用 `SHOW TABLES FROM database_name;` 指令檢視。

```sql
CREATE TABLE database_name.table_name (
    column_name_1 datatype,
    column_name_2 datatype,
    column_name_3 datatype,
    ....
);
```

In [6]:
CREATE TABLE cloned_imdb.movies (
    id int,
    title varchar(200),
    rating float,
    release_year year
);

## MySQL 的資料類型

- 整數。
- 浮點數。
- 文字。
- 日期與時間。

## MySQL 的整數資料類型

- `tinyint`: $-2^{7} \text{ 到 } 2^{7} - 1$（用 8-bit 儲存整數資料）
- `smallint`: $-2^{15} \text{ 到 } 2^{15} - 1$（用 16-bit 儲存整數資料）
- `mediumint`: $-2^{23} \text{ 到 } 2^{23} - 1$（用 24-bit 儲存整數資料）
- `int`: $-2^{31} \text{ 到 } 2^{31} - 1$（用 32-bit 儲存整數資料）
- `bigint`: $-2^{63} \text{ 到 } 2^{63} - 1$（用 64-bit 儲存整數資料）

## 設定是否要涵蓋小於零的負數

- 可以在整數資料類型後加上 `unsigned` 註明不涵蓋小於零的負數（無號整數）。
- 整數範圍可以多出一倍，例如 `tinyint` 為 -128 到 127，`tinyint unsigned` 會變為 0 到 255 

## 在資料類型為 `tinyint` 的欄位輸入 128 會出現錯誤

```
Data truncation: Out of range value for column 'id' at row 1
```

```sql
DROP TABLE IF EXISTS cloned_imdb.movies;
CREATE TABLE cloned_imdb.movies (
    id tinyint
);
INSERT INTO cloned_imdb.movies (id)
VALUES (128);
```

## 在資料類型為 `tinyint unsigned` 的欄位輸入 128 就會成功

In [7]:
DROP TABLE IF EXISTS cloned_imdb.movies;

In [8]:
CREATE TABLE cloned_imdb.movies (
    id tinyint unsigned
);

In [9]:
INSERT INTO cloned_imdb.movies (id)
VALUES (128);

## MySQL 的浮點數資料類型

- `float`：用 32-bit 儲存浮點數資料，有效位數到小數第 7 位左右。
- `double`：用 64-bit（`float` 的兩倍，故命名為 double precision）儲存浮點數資料，有效位數到小數第 15 位左右。

## 浮點數資料類型也能指定 `unsigned`

- 確保不會有負的數值輸入到該欄位。
- 但是不影響有效浮點數範圍。

```
Data truncation: Out of range value for column 'rating' at row 1
```

```sql
DROP TABLE IF EXISTS cloned_imdb.movies;
CREATE TABLE cloned_imdb.movies (
    id tinyint unsigned,
    rating float unsigned
);
INSERT INTO cloned_imdb.movies (id, rating)
VALUES (1, -9.2);
```

## MySQL 的文字資料類型

- `char`: 用 255-byte($2^{8} - 1$)儲存文字資料，固定長度。
- `varchar`: 用 65,535-byte($2^{16} - 1$)儲存文字資料，可變長度。
- `tinytext`: 用 255-byte($2^{8} - 1$)儲存文字資料，可變長度。
- `text`: 用 65,535-byte($2^{16} - 1$)儲存文字資料，可變長度。
- `mediumtext`: 用 16,777,215-byte($2^{24} - 1$)儲存文字資料，可變長度。
- `longtext`: 用 4,294,967,298-byte($2^{32} - 1$)儲存文字資料，可變長度。

## 如何選擇 MySQL 的文字資料類型

- 多數的狀態下使用 `varchar(最大字元數)` 即可。
- MySQL5.0 之後 `varchar` 用 65,535-byte 儲存文字資料，使得 `tinytext` 與 `text` 無用武之地。
- 如果儲存的是文件（例如部落格網站的資料庫），可以選擇 `mediumtext` 或 `longtext`。

## `char` 與 `varchar` 的差別

- `char` 為固定長度，會填入空白補滿長度。
- `varchar` 為可變長度，不會填入空白補滿長度。
- 使用固定長度的 `char` 能提高查詢、輸入資料的效率。

## 輸入超過字元長度的資料會出現錯誤

```
Data truncation: Out of range value for column 'id' at row 1
```

```sql
DROP TABLE IF EXISTS cloned_imdb.movies;
CREATE TABLE cloned_imdb.movies (
    id int unsigned,
    title varchar(20)
);
INSERT INTO cloned_imdb.movies (id, title)
VALUES (1, 'The Shawshank Redemption');
```

## MySQL 的日期時間資料類型

- `date`: 日期 `YYYY-MM-DD`
- `datetime`: 日期時間 `YYYY-MM-DD hh:mm:ss`
- `time`: 時間 `hh:mm:ss`
- `year`: 年 `YYYY`

## 輸入日期時間資料類型

- 只含有年份的資料直接以整數輸入。
- 年份以外的日期時間資料，以字元格式輸入。

In [10]:
DROP TABLE IF EXISTS cloned_imdb.movies;

In [11]:
CREATE TABLE cloned_imdb.movies (
    id int unsigned,
    title varchar(200),
    release_year year,
    release_date date
);

In [12]:
INSERT INTO cloned_imdb.movies (id, title, release_year, release_date)
VALUES (1, 'The Shawshank Redemption', 1994, '1994-10-14');

## 以 `ALTER TABLE` 更新資料表

- 以 `ADD` 新增欄位。
- 以 `DROP` 刪除欄位。
- 以 `RENAME` 重新命名欄位。
- 以 `MODIFY` 修改欄位資料類型。

## 新增欄位

```sql
ALTER TABLE database_name.table_name ADD column_name datatype;
```

In [13]:
DROP TABLE IF EXISTS cloned_imdb.movies;

In [14]:
CREATE TABLE cloned_imdb.movies (
    id int unsigned
);

In [15]:
ALTER TABLE cloned_imdb.movies ADD title varchar(200);

In [16]:
SELECT *
  FROM cloned_imdb.movies;

Empty set (0.00 sec)

## 刪除欄位

```sql
ALTER TABLE database_name.table_name DROP column_name;
```

In [17]:
ALTER TABLE cloned_imdb.movies DROP title;

In [18]:
SELECT *
  FROM cloned_imdb.movies;

Empty set (0.00 sec)

## 重新命名欄位

```sql
ALTER TABLE database_name.table_name RENAME COLUMN column_name TO new_column_name;
```

In [19]:
ALTER TABLE cloned_imdb.movies RENAME COLUMN id TO movie_id;

In [20]:
SELECT *
  FROM cloned_imdb.movies;

Empty set (0.00 sec)

## 修改欄位資料類型

```sql
ALTER TABLE database_name.table_name MODIFY COLUMN column_name new_datatype;
```

In [21]:
ALTER TABLE cloned_imdb.movies MODIFY COLUMN movie_id tinyint unsigned;

In [22]:
SHOW FIELDS FROM cloned_imdb.movies
           WHERE Field ='movie_id';

Field,Type,Null,Key,Default,Extra
movie_id,tinyint unsigned,YES,,NULL,


## 以 `TRUNCATE` 刪除資料表中的所有資料

```sql
TRUNCATE TABLE database_name.table_name;
```

In [23]:
DROP TABLE IF EXISTS cloned_imdb.movies;

In [24]:
CREATE TABLE cloned_imdb.movies (
    id int unsigned
);

In [25]:
INSERT INTO cloned_imdb.movies (id)
VALUES (1);

In [26]:
TRUNCATE TABLE cloned_imdb.movies;

In [27]:
SELECT *
  FROM cloned_imdb.movies;

Empty set (0.00 sec)

## 以 `DROP TABLE` 刪除資料表

```sql
DROP TABLE IF EXISTS database_name.table_name;
```

In [28]:
DROP TABLE IF EXISTS cloned_imdb.movies;

```sql
SELECT *
  FROM cloned_imdb.movies;
```

## 連線工具的使用者介面也可以輕鬆刪除資料表

左側資料庫按右鍵下拉式選單點選 Delete

## 檢視表（View）

## 什麼是檢視表

- 檢視表是一段被儲存在資料庫中的 SQL 敘述，具有一個物件命名。
- 想要檢視該段 SQL 敘述的查詢結果時，只需要將檢視表命名放在 `FROM` 保留字之後即可。

## 檢視表不等於資料表

- 實際上檢視表儲存的內容並不是列（Rows）與欄（Columns）所組成的二維表格，而是一段 SQL 敘述。
- 只有在對檢視表寫作資料查詢語言時，才會執行被儲存的 SQL 敘述。
- 簡單來說，檢視表是一種介於「子查詢」與「創造資料表」之間的功能。
- 檢視表就像是資料表版本的「衍生計算欄位」，由於多數「非資料庫管理員」的資料分析師在公司中沒有創造資料表的權限，因此若是有創造檢視表的權限將可以滿足我們對資料表的創造需求。

## 以 `CREATE VIEW` 創造檢視表

```sql
CREATE VIEW database_name.view_name
    AS
   SQL statement to be stored;
```

In [29]:
CREATE VIEW imdb.avg_rating_by_release_year
    AS
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM imdb.movies
 GROUP BY release_year;

## 將檢視表命名放在 `FROM` 保留字之後來使用

In [30]:
SELECT *
  FROM imdb.avg_rating_by_release_year
 WHERE avg_rating >= 8.7;

release_year,avg_rating
1994,8.8
1972,9.2
2023,8.85


## 以 `DROP VIEW` 刪除檢視表

```sql
DROP VIEW IF EXISTS database_name.view_name;
```

## 刪除檢視表之後，就無法在 `FROM` 保留字之後指定檢視表作為資料來源

```
Table 'imdb.avg_rating_by_release_year' doesn't exist
```

```sql
DROP VIEW IF EXISTS imdb.avg_rating_by_release_year;
SELECT *
  FROM imdb.avg_rating_by_release_year
 WHERE avg_rating >= 8.5;
```

## 連線工具的使用者介面也可以輕鬆刪除檢視表

左側資料庫按右鍵下拉式選單點選 Delete

## 透過檢視表複習 MySQL 不同的連接類型

- `JOIN` 保留兩個資料表的「交集」觀測值。
- `LEFT JOIN` 保留左資料表（接在 `FROM` 之後）的所有觀測值。
- `RIGHT JOIN` 保留右資料表（接在 `JOIN` 之後）的所有觀測值。

## 建立一個左檢視表 `casting_shawshank_darkknight`

In [31]:
CREATE VIEW imdb.casting_shawshank_darkknight
    AS
SELECT *
  FROM imdb.movies_actors
 WHERE movie_id IN (1, 3);

## 建立一個右檢視表 `movies_shawshank_forrest`

In [32]:
CREATE VIEW imdb.movies_shawshank_forrest
    AS
SELECT *
  FROM imdb.movies
 WHERE title IN ('The Shawshank Redemption', 'Forrest Gump');

## `JOIN` 保留兩個資料表的「交集」觀測值

In [33]:
SELECT movies_shawshank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM imdb.casting_shawshank_darkknight
  JOIN imdb.movies_shawshank_forrest
    ON casting_shawshank_darkknight.movie_id = movies_shawshank_forrest.id;

title,actor_id
The Shawshank Redemption,15237
The Shawshank Redemption,11321
The Shawshank Redemption,1673
The Shawshank Redemption,16177
The Shawshank Redemption,2758
The Shawshank Redemption,5408
The Shawshank Redemption,10334
The Shawshank Redemption,6901
The Shawshank Redemption,7232
The Shawshank Redemption,9235


## `LEFT JOIN` 保留左資料表的所有觀測值

In [34]:
SELECT movies_shawshank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM imdb.casting_shawshank_darkknight
  LEFT JOIN imdb.movies_shawshank_forrest
    ON casting_shawshank_darkknight.movie_id = movies_shawshank_forrest.id;

title,actor_id
The Shawshank Redemption,15237
The Shawshank Redemption,11321
The Shawshank Redemption,1673
The Shawshank Redemption,16177
The Shawshank Redemption,2758
The Shawshank Redemption,5408
The Shawshank Redemption,10334
The Shawshank Redemption,6901
The Shawshank Redemption,7232
The Shawshank Redemption,9235


## `RIGHT JOIN` 保留右資料表的所有觀測值

In [35]:
SELECT movies_shawshank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM imdb.casting_shawshank_darkknight
 RIGHT JOIN imdb.movies_shawshank_forrest
    ON casting_shawshank_darkknight.movie_id = movies_shawshank_forrest.id;

title,actor_id
The Shawshank Redemption,15237
The Shawshank Redemption,11321
The Shawshank Redemption,1673
The Shawshank Redemption,16177
The Shawshank Redemption,2758
The Shawshank Redemption,5408
The Shawshank Redemption,10334
The Shawshank Redemption,6901
The Shawshank Redemption,7232
The Shawshank Redemption,9235


## 角色（Role）與使用者（User）的創造

與資料控制語言（Data Control Language, DCL）相關，現在先略過不提。

## 重點整理

- 資料定義語言主要的保留字：
    - `CREATE`
    - `TRUNCATE`
    - `DROP`
    - `ALTER`

## 重點整理（續）

- 哪些資料庫管理系統中的物件可以被創造、刪除與更新：
    - 資料庫（Database）。
    - 資料表（Table）。
    - 檢視表（View）。
    - 角色（Role）與使用者（User）。

## 重點整理（續）

- MySQL 的資料類型
    - 整數。
    - 浮點數。
    - 文字。
    - 日期與時間。